In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
tfk = tf.keras
tf.keras.backend.set_floatx("float64")
import tensorflow_probability as tfp
tfd = tfp.distributions
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

In [ ]:
train = pd.read_csv('Final_Train.csv')

In [ ]:
test = pd.read_csv('Final_Test.csv')

In [ ]:
scaler = StandardScaler()
detector = IsolationForest(n_estimators=1000, behaviour="deprecated", contamination="auto", random_state=0)
neg_log_likelihood = lambda x, rv_x: -rv_x.log_prob(x)

In [ ]:
train.head()

,AGRA,AIZAWL,ALAPPUZHA,ALLAHABAD,ALWAR,AMRAVATI,AMRITSAR,BATHINDA,BHARUCH,BHOPAL,BULANDSHAHR,CACHAR,CHANDIGARH,CHENNAI,CHITTOOR,COIMBATORE,CUTTACK,DAKSHINAKANNADA,DEHRADUN,DHANBAD,DIBRUGARH,DURG,EASTKHASIHILLS,ERNAKULAM,FATEHGARHSAHIB,FIROZABAD,GANJAM,GHAZIABAD,GOLAGHAT,GUNTUR,GWALIOR,HASSAN,INDORE,JABALPUR,JAIPUR,JALANDHAR,JALGAON,JAMNAGAR,JHANSI,JODHPUR,...,PUNE,RAJKOT,RANCHI,RAYAGADA,SAGAR,SALEM,SANGLI,SIRMAUR,SOLAN,SOLAPUR,SONBHADRA,SONITPUR,SOUTHGOA,SURAT,THANE,THIRUVANANTHAPURAM,VADODARA,VALSAD,VARANASI,VISAKHAPATNAM,year,quarter,rainfall,relative_humidity,average_temperature,NSP Total/ NewMicConf Total,Relapse Total,NSN Total,so2,no2,rspm,Pediatric,Geriatric,15-24,25-34,35-44,45-54,55-64,NEP Total,SumOfHIV_infected
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2008,1,2.6,21.728250,18.952113,491,109,421,7.803030,23.744767,215.715909,24,26,111,102,101,73,54,275,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2008,2,150.8,35.571308,30.605233,795,127,551,7.341732,22.504294,184.648809,31,37,218,186,150,98,75,391,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2008,3,368.9,81.344806,28.749821,649,118,383,7.021000,17.422785,136.511628,30,30,190,132,131,84,52,363,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2008,4,0.8,54.863122,22.181595,686,126,346,6.832886,27.542857,224.551219,32,36,176,137,147,101,57,291,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2009,1,8.8,32.535027,20.001286,719,148,364,7.278723,21.356085,189.052632,38,33,165,137,165,101,80,378,0


In [ ]:
final = train.drop(['Pediatric' , 'Geriatric' , '15-24' , '25-34' , '35-44' , '45-54' , '55-64' , 'SumOfHIV_infected'] , axis = 1)

In [ ]:
final = final.fillna(method ='bfill') 

In [ ]:
data = final

In [ ]:
columns = data.columns

In [ ]:
X_t = scaler.fit_transform(data)

In [ ]:
is_inlier = detector.fit_predict(X_t)
X_t = X_t[(is_inlier > 0),:]

In [ ]:
dataset = pd.DataFrame(X_t, columns=columns)

In [ ]:
n_epochs = 50
n_samples = dataset.shape[0]
n_batches = 10
batch_size = np.floor(n_samples/n_batches)
buffer_size = n_samples

In [ ]:
inputs = ['AGRA', 'AIZAWL', 'ALAPPUZHA', 'ALLAHABAD', 'ALWAR', 'AMRAVATI',
       'AMRITSAR', 'BATHINDA', 'BHARUCH', 'BHOPAL', 'BULANDSHAHR', 'CACHAR',
       'CHANDIGARH', 'CHENNAI', 'CHITTOOR', 'COIMBATORE', 'CUTTACK',
       'DAKSHINAKANNADA', 'DEHRADUN', 'DHANBAD', 'DIBRUGARH', 'DURG',
       'EASTKHASIHILLS', 'ERNAKULAM', 'FATEHGARHSAHIB', 'FIROZABAD', 'GANJAM',
       'GHAZIABAD', 'GOLAGHAT', 'GUNTUR', 'GWALIOR', 'HASSAN', 'INDORE',
       'JABALPUR', 'JAIPUR', 'JALANDHAR', 'JALGAON', 'JAMNAGAR', 'JHANSI',
       'JODHPUR', 'KANPURNAGAR', 'KOLHAPUR', 'KOLLAM', 'KORBA', 'KOTTAYAM',
       'KOZHIKODE', 'KURNOOL', 'LATUR', 'LUCKNOW', 'LUDHIANA', 'MADURAI',
       'MORADABAD', 'NAGPUR', 'NASHIK', 'NORTHGOA', 'PALAKKAD', 'PATIALA',
       'PUNE', 'RAJKOT', 'RANCHI', 'RAYAGADA', 'SAGAR', 'SALEM', 'SANGLI',
       'SIRMAUR', 'SOLAN', 'SOLAPUR', 'SONBHADRA', 'SONITPUR', 'SOUTHGOA',
       'SURAT', 'THANE', 'THIRUVANANTHAPURAM', 'VADODARA', 'VALSAD',
       'VARANASI', 'VISAKHAPATNAM', 'year', 'quarter', 'rainfall',
       'relative_humidity', 'average_temperature',
       'NSP Total/ NewMicConf Total', 'Relapse Total', 'NSN Total', 'so2',
       'no2', 'rspm']

In [ ]:
outputs = ['NEP Total']

In [ ]:
n_train = int(0.7*dataset.shape[0])

In [ ]:
data = tf.data.Dataset.from_tensor_slices((dataset[inputs].values, dataset[outputs].values))
data = data.shuffle(n_samples, reshuffle_each_iteration=True)

In [ ]:
data_train = data.take(n_train).batch(batch_size).repeat(n_epochs)
data_test = data.skip(n_train).batch(1).repeat(n_epochs)

In [ ]:
prior = tfd.Independent(tfd.Normal(loc=tf.zeros(len(outputs), dtype=tf.float64), scale=1.0), reinterpreted_batch_ndims=1)

In [ ]:
model = tfk.Sequential([
tfk.layers.InputLayer(input_shape=(len(inputs),), name="input"),
tfk.layers.Dense(10, activation="relu", name="dense_1"),
tfk.layers.Dense(tfp.layers.MultivariateNormalTriL.params_size(
len(outputs)), activation=None, name="distribution_weights"),
tfp.layers.MultivariateNormalTriL(len(outputs), activity_regularizer=tfp.layers.KLDivergenceRegularizer(prior, weight=1/n_batches), name="output")
], name="model")

Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.


In [ ]:
model.compile(optimizer="adam", loss=neg_log_likelihood)

In [ ]:
model.fit(data_train, epochs=n_epochs, validation_data=data_test, verbose=False)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                890       
_________________________________________________________________
distribution_weights (Dense) (None, 2)                 22        
_________________________________________________________________
output (MultivariateNormalTr ((None, 1), (None, 1))    0         
Total params: 912
Trainable params: 912
Non-trainable params: 0
_________________________________________________________________


In [ ]:
tfp.layers.DenseFlipout(10, activation="relu", name="dense_1")

In [ ]:
samples = 500
iterations = 10
test_iterator = tf.compat.v1.data.make_one_shot_iterator(data_test)
X_true, Y_true, Y_pred = np.empty(shape=(samples, len(inputs))), np.empty(shape=(samples, len(outputs))), np.empty(shape=(samples, len(outputs), iterations))
for i in range(samples):
    features, labels = test_iterator.get_next()
    X_true[i,:] = features
    Y_true[i,:] = labels.numpy()
    for k in range(iterations):
        Y_pred[i,:,k] = model.predict(features)

In [ ]:
 Y_pred.shape

(500, 1, 10)

In [ ]:
Y_true.shape

(500, 1)

In [ ]:
pred = []

In [ ]:
for i in Y_pred:
  pred.append(i.mean())

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(Y_true , pred)

0.8838404244115738